<a href="https://colab.research.google.com/github/head1ton/Analysis_System/blob/main/character_chatbot/character_chatbot_development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import re
from datasets import Dataset

In [3]:
# Load Dataset

In [5]:
data_path = "/content/data/naruto.csv"
naruto_transcript_df = pd.read_csv(data_path)

In [6]:
naruto_transcript_df.head()

,name,line
0,Naruto,(Laughing) Give it up. (Shows the stone faces...
1,Hiruzen,(Turns away from his writing) I hope you’re n...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


In [7]:
# Remove actions from transcript
def remove_paranthesis(text):
    result = re.sub(r'\(.*?\)', '', text)
    return result

In [8]:
naruto_transcript_df['line'] = naruto_transcript_df['line'].apply(remove_paranthesis)

In [9]:
naruto_transcript_df.head()

,name,line
0,Naruto,"Give it up. You’re just bent, because you d..."
1,Hiruzen,I hope you’re not bothering me with some tri...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


In [10]:
naruto_transcript_df['number_of_words'] = naruto_transcript_df['line'].str.strip().str.split(" ")

In [11]:
naruto_transcript_df.head()

,name,line,number_of_words
0,Naruto,"Give it up. You’re just bent, because you d...","[Give, it, up., , You’re, just, bent,, because..."
1,Hiruzen,I hope you’re not bothering me with some tri...,"[I, hope, you’re, not, bothering, me, with, so..."
2,Ninja,Naseer Sabah,"[Naseer, Sabah]"
3,Ninja,is the best person on earth,"[is, the, best, person, on, earth]"
4,Naruto,muah,[muah]


In [12]:
naruto_transcript_df['number_of_words'] = naruto_transcript_df['number_of_words'].apply(lambda x: len(x))

In [13]:
naruto_transcript_df.head()

,name,line,number_of_words
0,Naruto,"Give it up. You’re just bent, because you d...",26
1,Hiruzen,I hope you’re not bothering me with some tri...,16
2,Ninja,Naseer Sabah,2
3,Ninja,is the best person on earth,6
4,Naruto,muah,1


In [14]:
naruto_transcript_df['naruto_response_flag'] = 0
naruto_transcript_df.loc[(naruto_transcript_df['name'] == "Naruto") & (naruto_transcript_df['number_of_words'] > 5), 'naruto_response_flag'] = 1

In [21]:
naruto_transcript_df[:10]

,name,line,number_of_words,naruto_response_flag
0,Naruto,"Give it up. You’re just bent, because you d...",26,1
1,Hiruzen,I hope you’re not bothering me with some tri...,16,0
2,Ninja,Naseer Sabah,2,0
3,Ninja,is the best person on earth,6,0
4,Naruto,muah,1,0
5,Iruka,"Oh yeah, Naruto!?",3,0
6,Naruto,"Where’d you come from, Iruka Sensei!? What ar...",11,1
7,Iruka,"No, what are you doing here? You’re supposed...",12,0
8,Iruka,"I’m at the end of my rope, Naruto. You faile...",43,0
9,Sakura,"Alright, Sakura here. Let’s do it. Transform!",7,0


In [16]:
indexes_to_take = list(naruto_transcript_df[(naruto_transcript_df['naruto_response_flag'] == 1) & (naruto_transcript_df.index > 0)].index)

In [19]:
indexes_to_take[:100]

[6,
 28,
 30,
 32,
 36,
 38,
 40,
 46,
 47,
 63,
 77,
 86,
 88,
 90,
 96,
 106,
 111,
 136,
 142,
 144,
 151]

In [22]:
system_promt = """" Your are Naruto from the anime "Naruto". Your responses should reflect his personality and speech patterns \n"""

prompts = []
for ind in indexes_to_take:
    prompt = system_promt

    prompt += naruto_transcript_df.iloc[ind - 1]['line']
    prompt += '\n'
    prompt += naruto_transcript_df.iloc[ind]['line']
    prompts.append(prompt)

In [23]:
print(prompts[3])

" Your are Naruto from the anime "Naruto". Your responses should reflect his personality and speech patterns 
 Well, I was just thinking. Maybe after you clean this all up I can take you out for some ramen. The good stuff. What do you think? 
  Now that’s some serious motivation! I’ll have this clean in no time!


In [24]:
df = pd.DataFrame({"prompt": prompts})
df.head()

,prompt
0,""" Your are Naruto from the anime ""Naruto"". You..."
1,""" Your are Naruto from the anime ""Naruto"". You..."
2,""" Your are Naruto from the anime ""Naruto"". You..."
3,""" Your are Naruto from the anime ""Naruto"". You..."
4,""" Your are Naruto from the anime ""Naruto"". You..."


In [25]:
dataset = Dataset.from_pandas(df)

In [27]:
dataset[:20]

{'prompt': ['" Your are Naruto from the anime "Naruto". Your responses should reflect his personality and speech patterns \n  Oh yeah, Naruto!?\n Where’d you come from, Iruka Sensei!? What are you doing here?',
  '" Your are Naruto from the anime "Naruto". Your responses should reflect his personality and speech patterns \n You’re not going home till you’ve cleaned off every single drop of paint.\n  So what? It’s not like there’s anyone waiting at home for me. ',
  '" Your are Naruto from the anime "Naruto". Your responses should reflect his personality and speech patterns \n  Naruto?\n  What do you want now, Sensei?',
  '" Your are Naruto from the anime "Naruto". Your responses should reflect his personality and speech patterns \n Well, I was just thinking. Maybe after you clean this all up I can take you out for some ramen. The good stuff. What do you think? \n  Now that’s some serious motivation! I’ll have this clean in no time!',
  '" Your are Naruto from the anime "Naruto". Your r